In [ ]:
%pip install -r requirements.txt

In [1]:
import os
import numpy as np
import time
import constructivo as constr
from auxiliar_functions import *
from local_search import *

#utilizar xlwt para escribir en excel


In [2]:
directory_path = '../JSSP_Instances'
directory_files = os.listdir(directory_path)
directory_files.sort()

In [3]:
import xlrd, xlwt
from xlutils.copy import copy as xl_copy
from xlwt import Workbook


def write_xlsx(machine_order, makespan, time, name_sheet, name_file):
    # Create a new workbook
    
    try:
        # open existing workbook
        rb = xlrd.open_workbook(name_file, formatting_info=True)
        # make a copy of it
        workbook = xl_copy(rb)
    except:
        workbook = Workbook()
    sheet = workbook.add_sheet(name_sheet)
    
    # Iterate over the matrix and write its values to the sheet
    for row_index, row in enumerate(machine_order, start=1):
        for col_index, value in enumerate(row, start=1):
            sheet.write(row_index - 1, col_index - 1, value)  # Note: row_index and col_index should start from 0
    
    # Write makespan and time to the next row
    row_index = len(machine_order)
    sheet.write(row_index, 0, int(makespan))
    sheet.write(row_index, 1, time)

    # Save the workbook to the specified file
    workbook.save(name_file)

In [4]:
with open('cotas.txt', 'r') as f:
    cotas = f.readlines()
    cotas = [int(line.strip()) for line in cotas]

with open('TimeLimit.txt', 'r') as f:
    # Note time is in minutes
    max_time = f.readlines()
    max_time = [int(line.strip()) for line in max_time]

In [5]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = cotas[file_num-1]

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

  # Record the start time
  vns_start_time = time.time()
  vns_sol = variable_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m, max_time=max_time[file_num-1])
  vns_end_time = time.time()

  print(file, "-> Constr: ", ans_constr, " Cota: ", cota_inferior)
  print("VNS -> " , makespan(vns_sol, processing_time, machines_required, n, m))
  print()
  

  write_xlsx(vns_sol, makespan(vns_sol , processing_time, machines_required, n, m), vns_end_time - vns_start_time, file, "JSSP_coberndorm_vns.xlsx")
  #write_xlsx(_, makespan(_, processing_time, machines_required, n, m), best_insert_back_end_time - best_insert_back_start_time, file, "JSSP_coberndorm_best_improvement_insert_backward.xlsx")


JSSP1.txt -> Constr:  1462  Cota:  991
VNS ->  1419

JSSP2.txt -> Constr:  1446  Cota:  985
VNS ->  1428

JSSP3.txt -> Constr:  1865  Cota:  1177
VNS ->  1845

JSSP4.txt -> Constr:  1667  Cota:  1218
VNS ->  1608

JSSP5.txt -> Constr:  2175  Cota:  1223
VNS ->  2082

JSSP6.txt -> Constr:  1965  Cota:  1270
VNS ->  1962

JSSP7.txt -> Constr:  2335  Cota:  1818
VNS ->  2291

JSSP8.txt -> Constr:  2432  Cota:  1838
VNS ->  2392

JSSP9.txt -> Constr:  2499  Cota:  1876
VNS ->  2405

JSSP10.txt -> Constr:  2710  Cota:  1784
VNS ->  2607

JSSP11.txt -> Constr:  3856  Cota:  2825
VNS ->  3507

JSSP12.txt -> Constr:  3266  Cota:  3001
VNS ->  3231

JSSP13.txt -> Constr:  3606  Cota:  2917
VNS ->  3547

JSSP14.txt -> Constr:  3639  Cota:  2950
VNS ->  3466

JSSP15.txt -> Constr:  6232  Cota:  5525
VNS ->  6152

JSSP16.txt -> Constr:  5973  Cota:  5676
VNS ->  5963



In [ ]:
with open('makespan_constr.txt', 'r') as f:
    makespan_constr = f.readlines()
    makespan_constr = [int(line.strip()) for line in makespan_constr]

In [181]:
import os
import xlrd

# Get a list of Excel files in the current directory
directory_files = os.listdir()
directory_files.sort()
directory_files = [file for file in directory_files if file.endswith('.xlsx')]

for file in directory_files:
    # Specify the path to your Excel file
    excel_file_path = file
    print("Current method: ", excel_file_path)
    average_distance_to_limit = []
    average_improvement = []

    for i in range(1,16+1):
        sheet_name = f"JSSP{i}.txt"  # Change this to the desired sheet name
        column_letter = "B"      # Change this to the desired column letter

        # Open the Excel workbook
        workbook = xlrd.open_workbook(excel_file_path, formatting_info=True)

        # Select the specific sheet by name
        sheet = workbook.sheet_by_name(sheet_name)

        # Find the last row with data in the specified column (column B)
        max_row = sheet.nrows -1

        # Read the value from the last cell in column B
        makespan_val = sheet.cell_value(max_row, 0)
        # Read the value from the last cell in column B
        time_val = sheet.cell_value(max_row, 1)

        cota_file = cotas[i-1]

        efficacy = (makespan_val - cota_file)/cota_file
        average_distance_to_limit.append(efficacy)
        average_improvement.append((makespan_constr[i-1] - makespan_val)/makespan_constr[i-1])

    print("average distance to limit: ", round(sum(average_distance_to_limit)/len(average_distance_to_limit), 4))
    print("average improvement: ", round(sum(average_improvement)/len(average_improvement),4))
    

Current method:  JSSP_coberndorm_vns.xlsx


NameError: name 'makespan_constr' is not defined

In [350]:
#Reading all the information
for file_num in range(1, len(directory_files)+1):
  file = "JSSP" + str(file_num) + ".txt"
  with open(directory_path + "/" + file, "r" ) as f:
  #with open("../AAA_test", "r" ) as f:
    size = f.readline().split()
    # n = trabajos, m = maquinas
    n, m = int(size[0]), int(size[1])
    data = list(f)
    for i,line in enumerate(data):
      data[i] = [int(x) for x in line.strip().split() if x is not None and x != '']

    processing_time = data[0:n]
    machines_required = data[n:2*n]
  
  # Cota inferior
  machines = {i:[[ j+1, processing_time[j][z], z, False] for z in range(m) for j in range(n)  if machines_required[j][z] == i]for i in range(1,m+1)}
  cota_inferior = cotas[file_num-1]

  # Create job objects
  jobs_constr = [constr.Job(np.array(machines_required)[i, :], np.array(processing_time)[i, :], i + 1) for i in range(n)]

  #Sorted array in which the first value is always the first machine that 
  machines_constr = [constr.Machine(i+1) for i in range(m)]
  # Record the start time
  constr_start_time = time.time(); ans_constr, machine_order_constr = constr.constructivo(jobs_constr, machines_constr); constr_end_time = time.time()

  # Record the start time
  vns_start_time = time.time()
  vns_sol = variable_neighborhood_search(machine_order_constr, processing_time, machines_required, n, m, max_time=max_time[file_num-1])
  vns_end_time = time.time()

  print(file, "-> Constr: ", ans_constr, " Cota: ", cota_inferior)
  print("VNS -> " , makespan(vns_sol, processing_time, machines_required, n, m))
  print()
  break

JSSP1.txt -> Constr:  1462  Cota:  991
VNS ->  1435



In [360]:
import random
import copy

def repair_method(destroyed_jobs, sol, machines_required, n, m):

    while True:

        new_sol = copy.deepcopy(sol)

        # Insert the shuffled values into the target list
        for mach in destroyed_jobs.keys():
            for job in destroyed_jobs[mach]:
                index_to_insert = random.randint(0, len(new_sol[mach-1]))
                new_sol[mach-1].insert(index_to_insert,job)

        return new_sol
        if feasible(new_sol, machines_required, n, m):
            return new_sol


def LNS(sol, processing_time, machines_required, n, m, lns_destroy_percentage):
    sol_time = makespan(sol, processing_time, machines_required, n, m)
    stop = False; job_size = n*m

    # Dictionary that maps the job number to the process number
    processes_num = {i * m + j +1: (i+1,machines_required[i][j])  for i in range(n) for j in range(m)}

    sol_time_best = sol_time

    while stop == False:
        stop = True
        
        sol_copy = copy.deepcopy(sol)
        # Implement Large Neighborhood Search (LNS)
        # Destroy a subset of the solution, e.g., by removing some jobs
        num_jobs_to_destroy = int(lns_destroy_percentage * job_size)
        destroyed_indices = random.sample(range(1,job_size+1), num_jobs_to_destroy)

        # Remove the jobs from the solution
        destroyed_jobs = {i:[] for i in range(1,m+1)}
        for index in destroyed_indices:
            job, mach = processes_num[index]
            destroyed_jobs[mach].append(job)
            sol_copy[mach-1].pop(sol_copy[mach-1].index(job))

        
        # Apply a repair method to reconstruct the solution
        repaired_sol = repair_method(destroyed_jobs, sol_copy, machines_required, n, m)

        return repaired_sol
        repaired_sol_time = makespan(repaired_sol, processing_time, machines_required, n, m)
        
        print(1)
        if repaired_sol_time < sol_time_best:
            sol_best = repaired_sol
            sol_time = repaired_sol_time
            stop = True
            return sol_best, sol_time
    
                    
    return sol

In [370]:
a = LNS(machine_order_constr, processing_time, machines_required, n, m, 0.1)
len(a), a

(15,
 [[12, 8, 10, 11, 9, 1, 4, 3, 15, 13, 2, 7, 5, 14, 6],
  [3, 15, 9, 7, 10, 8, 12, 11, 14, 2, 5, 6, 1, 13, 4],
  [12, 10, 4, 5, 1, 13, 9, 3, 8, 7, 6, 15, 14, 2, 11],
  [7, 6, 1, 10, 13, 8, 12, 3, 2, 15, 14, 11, 9, 5, 4],
  [2, 14, 11, 1, 5, 15, 7, 8, 6, 9, 3, 13, 12, 10, 4],
  [6, 13, 2, 8, 4, 14, 11, 3, 12, 9, 5, 7, 1, 10, 15],
  [10, 8, 1, 5, 9, 15, 3, 4, 12, 7, 14, 13, 11, 2, 6],
  [5, 9, 7, 2, 15, 11, 1, 12, 10, 3, 6, 14, 4, 13, 8],
  [14, 3, 10, 15, 7, 2, 5, 13, 12, 8, 11, 1, 9, 4, 6],
  [10, 3, 5, 4, 2, 11, 12, 13, 14, 15, 1, 8, 9, 7, 6],
  [15, 3, 9, 5, 13, 4, 12, 1, 10, 2, 7, 8, 11, 14, 6],
  [8, 15, 9, 3, 2, 1, 7, 6, 12, 14, 10, 5, 13, 4, 11],
  [7, 3, 15, 12, 8, 6, 1, 2, 11, 5, 9, 10, 14, 13, 4],
  [14, 2, 11, 10, 8, 3, 15, 6, 4, 12, 13, 1, 5, 7, 9],
  [14, 12, 2, 9, 5, 8, 15, 3, 1, 6, 11, 10, 4, 13, 7]])

In [382]:
a = [sorted(x) for x in machine_order_constr]
a

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]